In [1]:
from bs4 import BeautifulSoup, NavigableString, Tag
import urllib
import requests
import re
import nltk
import time

import pandas as pd

In [2]:
scr = []
# 101,576
for ep in range(1,80):
    url = "http://www.chakoteya.net/StarTrek/{}.htm".format(ep)
    try:
        scr.append(urllib.request.urlopen(url).read())
        time.sleep(3)
    except urllib.request.HTTPError as err:
        if err.code == 404:
            pass

In [6]:
many_soups = []
for ep in scr:
    many_soups.append(BeautifulSoup(ep, "lxml"))

In [7]:
many_soups[0].b.string

'The\r\nCage'

In [9]:
len(many_soups)

79

In [10]:
sent_tokenizer = nltk.tokenize.sent_tokenize
pattern = re.compile(r'(\b[A-Z]+|([A-Z]+.[A-Z]+))(\:|\s\[.+\]\:)')
df = pd.DataFrame(columns=['character', 'ep_title', 'text'])

In [11]:
for ep in many_soups:
    
    episode_title = ep.b.string
    episode_title = episode_title.replace('\r\n', ' ')
    
    array_of_strings = []
    
    for string in ep.stripped_strings:
        array_of_strings.append(string.replace('\r\n', ' '))
        
    clean_df = []
    char_dict = {}

    for string in array_of_strings:
        found = re.search(pattern, string)
        if found is not None:
            stripped_string = string.replace(found.group(0), '').strip()
            stripped_string_tokenized = sent_tokenizer(stripped_string)

            key = found.group(1)

            for each in stripped_string_tokenized:
                    clean_df.append(each)
                    char_dict.setdefault(key, []).append(each)
    
    for key in char_dict:
        temp_df = pd.DataFrame(char_dict[key], columns=['text'])
        temp_df['character'] = key
        temp_df['ep_title'] = episode_title
        df = df.append(temp_df)

In [12]:
df.head()

,character,ep_title,text
0,SPOCK,The Cage,Check the circuit.
1,SPOCK,The Cage,It can't be the screen then.
2,SPOCK,The Cage,"Definitely something out there, Captain, heade..."
3,SPOCK,The Cage,Their call letters check with a survey expedit...
4,SPOCK,The Cage,SS Columbia.


In [13]:
df.describe()

,character,ep_title,text
count,54047,54047,54047
unique,436,79,45795
top,KIRK,A Piece Of The Action,Yes.
freq,17001,882,287


In [14]:
df.to_csv('dataframe_all_tos.csv', index=False)

In [15]:
df_test = pd.read_csv('./dataframe_all_tos.csv')

In [16]:
df_test.head()

,character,ep_title,text
0,SPOCK,The Cage,Check the circuit.
1,SPOCK,The Cage,It can't be the screen then.
2,SPOCK,The Cage,"Definitely something out there, Captain, heade..."
3,SPOCK,The Cage,Their call letters check with a survey expedit...
4,SPOCK,The Cage,SS Columbia.
